<a href="https://colab.research.google.com/github/everfgmolinas/TSP_IA/blob/main/TSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TSP

### Librerias

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import time
import warnings
warnings.filterwarnings('ignore')

### Constantes

In [2]:
# Definicion de constantes a ser utilizadas

# maximo valor aleatorio que se podrá generar
valorMaximo = 100
# numero de nodos a ser utilizados
nodos = 8

### Generancion del grafo y punto inicial

In [3]:
# el par (nodos,2) da las dimensiones del array que generará numpy (fila,col)
puntosAleatorios = pd.DataFrame(
    np.random.randint(0, valorMaximo, (nodos,2)),
    columns=['X', 'Y']
)


puntosAleatorios.nunique()

# desde que arista empezar
inicio = np.random.randint(0, nodos-1)

### 1. Funciones utiles

##### 1.2 Definicion de funcion que calcula la distancia entre dos puntos

In [4]:
def distancia( punto1, punto2):
  return np.sqrt(np.power(punto1[0]-punto2[0],2) + np.power(punto1[1]-punto2[1],2))

##### 1.3 Definicion de funcion que retorna la cantidad de nodos expandidos

In [5]:
def estadosExpandidos(grafo):
    # contamos el numero de nodos visitados
    return grafo[grafo['marcado']==True].shape[0]

##### 1.4 Definicion de funcion que encuentra el punto más cercano respecto a un parámetro dado

In [6]:
# busqueda de siguiente distancia menor
# si no encuentra a nadie con una menor distancia y sin visitar, devuelve el mismo 
# punto desde el que se busca
def punto_mas_cerca(df, punto_inicial):
  """Determina el siguiente punto más cercano con respecto al punto_inicial,
  si no encuentra un punto se retorna el mismo.

  Parameters
  ----------
    df : Dataframe de puntos
    punto_inicial : punto a partir de donde se busca el siguiente punto

  Returns
  ----------

    df.row : fila del dataframe que representa el punto mascercano
    int : longitud con el punto mas cercano o numpy.inf
  """

  # longitud infinita
  min_lenght = np.inf

  # el siguiente punto más cercano es si mismo
  next = punto_inicial.name

  for index, row in df.iterrows():
    # no debe de haber sido visitado y debe de ser una longitud menor a la
    # longitud minima
    if(row[2] != True and distancia(row, punto_inicial) <= min_lenght):
      next = index
      min_lenght = distancia(row, punto_inicial)

  # retorna el punto y la longitud
  return df.iloc[next], min_lenght

##### 1.5 Definicion de funcion que verifica la existencia de un nodo en un grafo

In [7]:
def existe_en_dataframe(df, data):
  query = "X == " + str(data['X']) + " and Y == " + str(data['Y'])
  return len(df.query(query)) >= 1

##### 1.6 Definicion de funcion encargada de encontrar la ruta óptima aplicando el mas avaro

In [8]:
def rutaOptima(dataframe, inicio):
  dataframeCP = dataframe.copy()
  # la columna marcado funciona para ver si el nodo en cuestion ya fue visitado
  dataframeCP = dataframeCP.assign(marcado=False)
  # cambiar a visitado
  dataframeCP.iloc[inicio,2] = True
  ruta = {
    'ruta': pd.DataFrame(columns = ['X' , 'Y']),
    'distancia': 0
  }

  ruta['ruta'] = ruta['ruta'].append({'X': dataframeCP.iloc[inicio].X, 'Y':dataframeCP.iloc[inicio].Y},ignore_index=True)

  # mientras no haya un punto sin visitar
  while(dataframeCP.marcado.isin([False]).any()):

    #obtener punto mas cercano
    punto_siguiente, distancia_sig = punto_mas_cerca(dataframeCP, ruta['ruta'].iloc[-1])
    dataframeCP.iloc[punto_siguiente.name,2] = True
    ruta['ruta'] = ruta['ruta'].append({'X': punto_siguiente.X, 'Y':punto_siguiente.Y},ignore_index=True)
    ruta['distancia']+= distancia_sig

  #conectar con el inicio
  ruta['ruta'] = ruta['ruta'].append({'X': ruta['ruta'].iloc[0].X, 'Y':ruta['ruta'].iloc[0].Y},ignore_index=True)
  ruta['distancia']+= distancia(ruta['ruta'].iloc[-1], ruta['ruta'].iloc[-2])
  
  return ruta

##### 1.7 Definicion de funcion encargada de invertir el orden de un dataframe

In [9]:
def invertir(dataframe, inicio, fin):
    """
    Invierte desde inicio + 1, hasta fin
    """
    return pd.concat([dataframe.iloc[:inicio+1],dataframe[inicio+1:fin+1:][::-1],dataframe.iloc[fin+1:]])

In [10]:
def distancia2( punto1, punto2):
  return np.power(punto1[0]-punto2[0],2) + np.power(punto1[1]-punto2[1],2)

In [11]:
def puntos_mas_cercanos(df,df2, punto_inicial):
  """Determina los siguientes puntos más cercanos con respecto al punto_inicial,
  si no encuentra un punto se retorna el mismo.

  Parameters
  ----------
    df : Dataframe de puntos totales
    df2 : Dataframe de puntos ya utilizados
    punto_inicial : punto a partir de donde se busca el siguiente punto

  Returns
  ----------

    array de df.row : array de puntos mas cercanos
    int : longitud con el punto mas cercano o numpy.inf
  """

  # longitud infinita
  min_lenght = np.inf

  # el siguiente punto más cercano es si mismo
  next = [punto_inicial]

  for index, row in df.iterrows():
    # no debe de existir en el segundo dataframe y debe de ser una longitud menor a la
    # longitud minima
    if(existe_en_dataframe(df2,row) != True and distancia(row, punto_inicial) <= min_lenght):
      if (distancia(row, punto_inicial) < min_lenght):
        next = [row]
        min_lenght = distancia(row, punto_inicial)
      else:
        next.append(row)

  # retorna el punto y la longitud
  return next, min_lenght

##### 1.6 Ejemplo de medición de tiempo 

In [12]:
# COMO MEDIR EL TIEMPO DE EJECUCION

# capturamos la hora de inicializacion
inicio_time = time.time()

# aqui deberiamos agregar cada una de las llamadas a las funciones
# como ejemplo una operacion simple
y = 25*3

# capturamos la hora de finalizacion
fin_time = time.time()

print(fin_time-inicio_time)

4.4345855712890625e-05


## 2. TSP con Backtracking

### 2.1 Funciones

In [13]:
def search(ruta_optima, ruta, grafo, inicio):
  for h in range(0,len(grafo)):
    punto = {"X": grafo.iloc[h].X, "Y": grafo.iloc[h].Y}
    if(not existe_en_dataframe(ruta['ruta'], punto)):
      ruta['ruta'] = ruta['ruta'].append(punto, ignore_index = True)
      ruta['distancia'] += distancia(ruta['ruta'].iloc[-1], ruta['ruta'].iloc[-2])
      ruta_optima = search(ruta_optima, ruta, grafo, inicio)
      ruta['distancia'] -= distancia(ruta['ruta'].iloc[-1], ruta['ruta'].iloc[-2])
      ruta['ruta'] = ruta['ruta'].drop([len(ruta['ruta'])-1], axis=0)

  #conectar con el primer punto solo si la ruta ya fue totalmente recorrida
  if ( len(ruta['ruta']) == len(grafo) ):
    punto = {"X": grafo.iloc[inicio].X, "Y": grafo.iloc[inicio].Y}
    ruta['ruta'] = ruta['ruta'].append(punto, ignore_index = True)
    ruta['distancia'] += distancia(ruta['ruta'].iloc[-1], ruta['ruta'].iloc[-2])

  if (ruta['distancia'] <= ruta_optima['distancia'] and len(ruta['ruta']) == (len(grafo)+1)) :
    if(ruta['distancia'] < ruta_optima['distancia']):

      ruta_optima['ruta'] = [ruta['ruta'].copy()]
      ruta_optima['distancia'] = ruta['distancia']

    elif(ruta['distancia'] == ruta_optima['distancia']):

      ruta_optima['ruta'].append(ruta['ruta'].copy())
      ruta_optima['distancia'] = ruta['distancia']

  if(len(ruta['ruta']) == (len(grafo)+1)):
    #borrar la conexion con el inicio
    ruta['distancia'] -= distancia(ruta['ruta'].iloc[-1], ruta['ruta'].iloc[-2])
    ruta['ruta'] = ruta['ruta'].drop([len(ruta['ruta'])-1], axis=0)
    
  return ruta_optima

In [14]:
def backtracking(dataframe, inicio):
  ruta = {
    'ruta': pd.DataFrame(columns = ['X' , 'Y']),
    'distancia': 0
  }
  ruta_optima = {
    'ruta': [],
    'distancia': np.inf
  }
  ruta['ruta'] = ruta['ruta'].append({'X': dataframe.iloc[inicio].X, 'Y':dataframe.iloc[inicio].Y},ignore_index=True)
  return search(ruta_optima, ruta, dataframe, inicio)

## 3. TSP con heuristica

### 3.1 Funciones

In [15]:
def ruta2Optima(dataframe, inicio):
  # realizar una copia para no afectar al original
  dataframeCP = dataframe.copy()
    
  # crear un diccionario
  ruta = {
    'ruta': pd.DataFrame(columns = ['X' , 'Y']),  # dataframe de puntos que representan el orden del recorrido
    'distancia': 0
  }

  # agregamos el punto inicial
  ruta['ruta'] = ruta['ruta'].append({'X': dataframeCP.iloc[inicio].X, 'Y':dataframeCP.iloc[inicio].Y},ignore_index=True)

  # mientras no haya un punto sin visitar
  while(len(ruta['ruta'])!= len(dataframe)):
        
    #obtener puntos mas cercanos
    puntos_siguientes, distancia_sig = puntos_mas_cercanos(dataframeCP,ruta['ruta'], ruta['ruta'].iloc[-1])
    
    # array de heuristicas
    heuristicas = []
    
    # generar heuristicas por cada punto posible
    for punto in puntos_siguientes:
        
        # nuevo se hace una copia del estado actual
        ruta_cp = ruta.copy()
        
        # se agrega el punto cercano al estado obteniendo asi el siguiente estado a analizar
        ruta_cp['ruta'] = ruta_cp['ruta'].append({'X': punto.X, 'Y':punto.Y},ignore_index=True)
        ruta_cp['distancia']+= distancia_sig
        
        # se calcula la conveniencia del estado de acuerto a la heuristica 2opt y se agrega a las heuristicas
        heuristicas.append(opt2_local(ruta_cp, len(ruta_cp['ruta'])-2))
        
    # la mejor heuristica es aquella que se desconoce
    heuristica = np.inf
    
    # buscar por cada heuristica
    for ruta_2opt in heuristicas:
        
        # la heuristica es mejor que la que  tenemos
        if(ruta_2opt['distancia']<heuristica):
            
            # actualizamos el valor de la heuristica
            heuristica = ruta_2opt['distancia']
            
            # esa heuristica sera nuestro proximo estado
            ruta = ruta_2opt
            
  #conectar con el inicio
  ruta['ruta'] = ruta['ruta'].append({'X': ruta['ruta'].iloc[0].X, 'Y':ruta['ruta'].iloc[0].Y},ignore_index=True)
  ruta['distancia']+= distancia(ruta['ruta'].iloc[-1], ruta['ruta'].iloc[-2])
  
  # aplicar la optimizacion al volver al inicio
  return opt2_local(ruta, len(ruta_cp['ruta'])-2)

In [16]:
def opt2_local(ruta, fin):
    """optimizacion local por 2opt,

    Parameters
    ----------
    ruta : ruta la cual optimizar
    fin: hasta que punto hacer la optimizacion

    Returns
    ----------

    ruta optimizada
    """
    foundImprovement = True
    
    # mientras se encuentre una optmimizacion se vuelve a analizar desde el principio debido a un cambio de ruta
    while (foundImprovement):
        foundImprovement = False;

        # desde el inicio hasta el final
        for i in range(fin-1):
            
            # calcular la mejora de distancia si se incercambia dos aristas
            lengthDelta = - distancia(ruta['ruta'].iloc[i],ruta['ruta'].iloc[i+1]) - distancia(ruta['ruta'].iloc[fin],ruta['ruta'].iloc[fin+1])+ distancia(ruta['ruta'].iloc[i],ruta['ruta'].iloc[fin])+ distancia(ruta['ruta'].iloc[i+1],ruta['ruta'].iloc[fin+1])

            # si la distancia es negativa, significa una mejora
            if (lengthDelta < 0) :
                
                # invertimos la ruta entre "i+1" e "fin"
                ruta['ruta'] = invertir(ruta['ruta'], i, fin)
                
                # aplicamos la mejora de la ruta
                ruta['distancia'] += lengthDelta;
                
                # anuncimamos un cambio en la ruta
                foundImprovement = True;
    return ruta

## Pruebas varias

### Pruebas de distancia entre puntos

In [17]:
distancia(puntosAleatorios.iloc[inicio], puntosAleatorios.iloc[inicio+1])

44.82186966202994

### Pruebas ruta con mas avaro

In [18]:
inicio_time_avaro = time.time()

ruta_optima = rutaOptima(puntosAleatorios, inicio)

# capturamos la hora de finalizacion
fin_time_avaro = time.time()

print(fin_time_avaro-inicio_time_avaro)

0.049463510513305664


### Pruebas ruta con backtracking

In [ ]:
inicio_time_back = time.time()

ruta_backtracking = backtracking(puntosAleatorios, inicio)
# capturamos la hora de finalizacion
fin_time_back = time.time()

print(fin_time-inicio_time)

### Pruebas ruta con heuristica

In [ ]:
inicio_time = time.time()

ruta_optima2opt = ruta2Optima(puntosAleatorios, inicio)
# capturamos la hora de finalizacion
fin_time = time.time()

print(fin_time-inicio_time)

## Graficos

In [ ]:
fig = px.line(ruta_optima['ruta'], x="X", y="Y", title='Grafico con Mas avaro')
fig.show()

In [ ]:
for ruta in ruta_backtracking['ruta'] :
  fig = px.line(ruta, x="X", y="Y", title='Grafico con Backtracking', text=ruta.index)
  fig.update_traces(textposition="bottom right")
  fig.show()

In [ ]:
fig = px.line(ruta_optima2opt['ruta'], x="X", y="Y", title='Ruta con heuristica', text=ruta_optima2opt['ruta'].index)
fig.show()